In [16]:
import pandas
import pickle
import datetime
import time
import math
import statistics
import pytz
import numpy as np
import alpaca_trade_api as api

In [17]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', None)
pandas.set_option('display.float_format', lambda x: '%.3f' % x)

In [18]:
API_KEY = 'AKCBVDALZTXXE0NJ7LMV'
SECRET_KEY = 'OgCPIyKsZ9g1iLrmU89uKcWUCPW59E8Vrc6bGzwz'


In [19]:
alpaca = api.REST(API_KEY, SECRET_KEY)

In [20]:
dt_format = '%Y-%m-%d %H:%M:%S'

start_date_string = '2020-12-10 00:00:00'
end_date_string = '2022-12-14 23:59:59'

start_dt = pytz.timezone('US/Eastern').localize(datetime.datetime.strptime(start_date_string, dt_format)).astimezone(pytz.utc).isoformat()
end_dt = pytz.timezone('US/Eastern').localize(datetime.datetime.strptime(end_date_string, dt_format)).astimezone(pytz.utc).isoformat()



In [21]:
print(start_dt)
print(end_dt)

2020-12-10T05:00:00+00:00
2022-12-15T04:59:59+00:00


In [22]:
bars = alpaca.get_bars("SPY", api.TimeFrame(1, api.TimeFrameUnit.Minute), "2020-10-15", "2023-01-30", adjustment='raw').df

In [23]:
def compute_fuzzy_mean(series, days_lookback=5, days_lookback_buffer=5):
    ttl_lookback_distance = days_lookback + days_lookback_buffer
    
    samples = [volume for i in range(1, ttl_lookback_distance + 1) if not math.isnan((volume := series[f'dod_volume_{i}']))]
    
    # downselect to desired number of samples
    samples = samples[:days_lookback]
    
    count = len(samples)
    mean = statistics.fmean(samples) if count > 0 else float('nan')
    stddev = statistics.pstdev(samples) if count > 1 else float('nan')
    
    dod_volume_mean_column_name = f'dod_volume_mean_{days_lookback}'
    dod_volume_stddev_column_name = f'dod_volume_stddev_{days_lookback}'
    dod_volume_mean_count_column_name = f'dod_volume_mean_count_{days_lookback}'
    
    return {dod_volume_mean_column_name: mean, dod_volume_stddev_column_name: stddev, dod_volume_mean_count_column_name: count}


def compute_rvol(df, days_lookback=5, days_lookback_buffer=10, rvol_sma_minutes=15):
    # Don't perform inplace
    df = df.copy()
    
    ttl_lookback_distance = days_lookback + days_lookback_buffer
    for i in range(1, ttl_lookback_distance + 1):
        column_name = f'dod_volume_{i}'
        time_delta = pandas.to_timedelta(i, 'd')
        df[column_name] = df.apply(lambda x: df.loc[x.name - time_delta].volume if (x.name - time_delta) in df.index else float('nan'), axis=1)

    mean_stddev_count_df = df.apply(lambda x: compute_fuzzy_mean(x, days_lookback, days_lookback_buffer), result_type='expand', axis=1)
    
    df = pandas.concat([df, mean_stddev_count_df], axis='columns')
    
    # Compute and set rvol
    rvol_column_name = f'rvol_{days_lookback}'
    rvol_z_score_column_name = f'rvol_z_score_{days_lookback}'
    df[rvol_column_name] = df['volume'] / df[f'dod_volume_mean_{days_lookback}']
    df[rvol_z_score_column_name] = (df['volume']  - df[f'dod_volume_mean_{days_lookback}'])/df[f'dod_volume_stddev_{days_lookback}']
    
    # Compute and set rvol moving averages
    rvol_sma_column_name = f'{rvol_column_name}_SMA_{rvol_sma_minutes}min'
    rvol_z_score_sma_column_name = f'{rvol_z_score_column_name}_SMA_{rvol_sma_minutes}min'
    df[rvol_sma_column_name] = df[rvol_column_name].rolling(f'{rvol_sma_minutes}min', min_periods=1).mean()
    df[rvol_z_score_sma_column_name] = df[rvol_z_score_column_name].rolling(f'{rvol_sma_minutes}min', min_periods=1).mean()
    
    # Drop columns that are no longer needed
    dod_volume_mean_column_name = f'dod_volume_mean_{days_lookback}'
    dod_volume_stddev_column_name = f'dod_volume_stddev_{days_lookback}'
    dod_volume_mean_count_column_name = f'dod_volume_mean_count_{days_lookback}'
    df = df[['open', 'high', 'low', 'close', 'volume', 'trade_count', dod_volume_mean_column_name, dod_volume_stddev_column_name, dod_volume_mean_count_column_name, rvol_column_name, rvol_z_score_column_name, rvol_sma_column_name, rvol_z_score_sma_column_name]]
    
    return df


In [24]:
bars_output = compute_rvol(bars)

In [25]:
bars_output.tail(5)

,open,high,low,close,volume,trade_count,dod_volume_mean_5,dod_volume_stddev_5,dod_volume_mean_count_5,rvol_5,rvol_z_score_5,rvol_5_SMA_15min,rvol_z_score_5_SMA_15min
timestamp,,,,,,,,,,,,,
2023-01-31 00:54:00+00:00,401.360,401.360,401.360,401.360,850,13,3327.400,4077.712,5.000,0.255,-0.608,0.922,0.079
2023-01-31 00:56:00+00:00,401.370,401.400,401.370,401.400,2860,22,4215.600,5173.950,5.000,0.678,-0.262,0.900,0.053
2023-01-31 00:57:00+00:00,401.400,401.450,401.400,401.450,2315,31,1502.400,1327.994,5.000,1.541,0.612,0.962,0.120
2023-01-31 00:58:00+00:00,401.390,401.410,401.390,401.410,1755,30,5584.000,5882.222,5.000,0.314,-0.651,0.960,0.114
2023-01-31 00:59:00+00:00,401.410,401.440,401.410,401.440,1625,32,16317.000,19056.025,5.000,0.100,-0.771,0.964,0.094


In [26]:
def populate_rvol(trades_df, rvol_complete_bars_df, rvol_days_lookback=5, rvol_sma_minutes=15):
    trades_df = trades_df.copy()
    
    # All columns we want to map from rvol_complete_bars_df
    dod_volume_mean_column_name = f'dod_volume_mean_{rvol_days_lookback}'
    dod_volume_stddev_column_name = f'dod_volume_stddev_{rvol_days_lookback}'
    dod_volume_mean_count_column_name = f'dod_volume_mean_count_{rvol_days_lookback}'
    rvol_column_name = f'rvol_{rvol_days_lookback}'
    rvol_z_score_column_name = f'rvol_z_score_{rvol_days_lookback}'
    rvol_sma_column_name = f'rvol_{rvol_days_lookback}_SMA_{rvol_sma_minutes}min'
    rvol_z_score_sma_column_name = f'rvol_z_score_{rvol_days_lookback}_SMA_{rvol_sma_minutes}min'
    
    # Downselect only these columns
    rvol_complete_bars_df = rvol_complete_bars_df[[dod_volume_mean_column_name, dod_volume_stddev_column_name, dod_volume_mean_count_column_name, rvol_column_name, rvol_z_score_column_name, rvol_sma_column_name, rvol_z_score_sma_column_name]]
    
    # Compute entry_time_minute
    trades_df['entry_time_previous_minute'] = trades_df.apply(lambda x: x['entry_time'].replace(second=0, microsecond=0, nanosecond=0) - pandas.Timedelta(minutes=1), axis=1)
    
    # Merge in desired columns
    trades_with_rvol = pandas.merge(trades_df, rvol_complete_bars_df, how='left', left_on = 'entry_time_previous_minute', right_index = True)
    
    # Drop columns that are no longer needed
#     trades_with_rvol.drop(columns=['entry_time_previous_minute'], inplace=True)
    
    return trades_with_rvol


# TEST populate_rvol

In [27]:
bars_output = compute_rvol(bars)
rt = pandas.read_pickle("../output/trades.pickle")
_df = populate_rvol(rt, bars_output, rvol_days_lookback=5, rvol_sma_minutes=15)

In [28]:
_df.head()

,parameter_buffer,parameter_stop,parameter_target,target_r,distance,direction,date,entry_price,entry_order_price,exit_price,stop_hit,target_hit,time_close,target_price,stop_price,max_target_exit_price,entry_time,max_target_exit_time,max_distance,max_r,theoretical_r,r,entry_time_previous_minute,dod_volume_mean_5,dod_volume_stddev_5,dod_volume_mean_count_5,rvol_5,rvol_z_score_5,rvol_5_SMA_15min,rvol_z_score_5_SMA_15min
0,0.000,0.200,0.200,1.000,1.330,short,2021-12-14,464.420,464.420,464.220,False,True,False,464.220,464.620,464.160,2021-12-14 14:38:37.218842448+00:00,2021-12-14 14:39:53.141943552+00:00,0.260,1.300,1.000,1.000,2021-12-14 14:37:00+00:00,245931.600,95038.621,5.000,1.156,0.402,1.212,0.774
1,0.000,0.200,0.300,1.500,1.330,short,2021-12-14,464.420,464.420,464.620,True,False,False,464.120,464.620,464.160,2021-12-14 14:38:37.218842448+00:00,2021-12-14 14:39:53.141943552+00:00,0.260,1.300,-1.000,-1.000,2021-12-14 14:37:00+00:00,245931.600,95038.621,5.000,1.156,0.402,1.212,0.774
2,0.000,0.200,0.400,2.000,1.330,short,2021-12-14,464.420,464.420,464.620,True,False,False,464.020,464.620,464.160,2021-12-14 14:38:37.218842448+00:00,2021-12-14 14:39:53.141943552+00:00,0.260,1.300,-1.000,-1.000,2021-12-14 14:37:00+00:00,245931.600,95038.621,5.000,1.156,0.402,1.212,0.774
3,0.000,0.200,0.500,2.500,1.330,short,2021-12-14,464.420,464.420,464.620,True,False,False,463.920,464.620,464.160,2021-12-14 14:38:37.218842448+00:00,2021-12-14 14:39:53.141943552+00:00,0.260,1.300,-1.000,-1.000,2021-12-14 14:37:00+00:00,245931.600,95038.621,5.000,1.156,0.402,1.212,0.774
4,0.000,0.200,0.600,3.000,1.330,short,2021-12-14,464.420,464.420,464.620,True,False,False,463.820,464.620,464.160,2021-12-14 14:38:37.218842448+00:00,2021-12-14 14:39:53.141943552+00:00,0.260,1.300,-1.000,-1.000,2021-12-14 14:37:00+00:00,245931.600,95038.621,5.000,1.156,0.402,1.212,0.774


# Test different values for days_lookback

In [70]:
bars = alpaca.get_bars("SPY", api.TimeFrame(1, api.TimeFrameUnit.Minute), "2021-10-14", "2022-12-15", adjustment='raw').df

In [71]:
rt = pandas.read_pickle("../output/trades_y2.pickle")

In [72]:
df = rt
df = df[(df['parameter_buffer'] == 0.4) & (df['parameter_stop'] == 0.2) & (df['parameter_target'] == 0.2) & (df['distance'] > 0.75)]
df = df[df['direction'] == 'long']
print(df.shape)

(107, 22)


In [73]:
df_good = df[df['r'] > 0]
df_bad = df[df['r'] < 0]
print(f'df_good - {(df_good.shape[0] / df.shape[0]):.2%}')
print(f' df_bad - {(df_bad.shape[0] / df.shape[0]):.2%}')

df_good - 62.62%
 df_bad - 37.38%


In [78]:
days_lookbacks = [6, 7]
threshold_start = 0.5
threshold_end = 4
threshold_step = 0.1
for days_lookback in days_lookbacks:
    start = time.time()
    print(f'PROCESSING days_lookback -> {days_lookback}')
    bars_output = compute_rvol(bars, days_lookback=days_lookback, days_lookback_buffer=days_lookback*2, rvol_sma_minutes=15)
    _df = populate_rvol(df, bars_output, rvol_days_lookback=days_lookback, rvol_sma_minutes=15)
    df_good = _df[_df['r'] > 0]
    df_bad = _df[_df['r'] < 0]

    threshold = threshold_start
    while threshold < threshold_end:
        c1 = f'rvol_{days_lookback}_SMA_15min'
        _df_good = df_good[df_good[c1] < threshold]
        _df_bad = df_bad[df_bad[c1] < threshold]
        print(f'{threshold:0.1f}:  {(_df_good.shape[0] / (_df_good.shape[0] + _df_bad.shape[0])):.2%} vs {(_df_bad.shape[0] / (_df_good.shape[0] + _df_bad.shape[0])):.2%},   {(_df_good.shape[0] + _df_bad.shape[0]):3} ttl')
        threshold = round(threshold + threshold_step, 2)
    
    print(f'      - {(time.time() - start):.1f} seconds elapsed')


PROCESSING days_lookback -> 6
0.5:  66.67% vs 33.33%,     3 ttl
0.6:  50.00% vs 50.00%,     4 ttl
0.7:  50.00% vs 50.00%,     8 ttl
0.8:  57.89% vs 42.11%,    19 ttl
0.9:  68.42% vs 31.58%,    38 ttl
1.0:  73.58% vs 26.42%,    53 ttl
1.1:  71.64% vs 28.36%,    67 ttl
1.2:  72.22% vs 27.78%,    72 ttl
1.3:  71.43% vs 28.57%,    77 ttl
1.4:  70.73% vs 29.27%,    82 ttl
1.5:  70.93% vs 29.07%,    86 ttl
1.6:  70.00% vs 30.00%,    90 ttl
1.7:  68.48% vs 31.52%,    92 ttl
1.8:  68.09% vs 31.91%,    94 ttl
1.9:  68.09% vs 31.91%,    94 ttl
2.0:  68.09% vs 31.91%,    94 ttl
2.1:  67.37% vs 32.63%,    95 ttl
2.2:  67.01% vs 32.99%,    97 ttl
2.3:  66.67% vs 33.33%,    99 ttl
2.4:  65.35% vs 34.65%,   101 ttl
2.5:  65.35% vs 34.65%,   101 ttl
2.6:  64.71% vs 35.29%,   102 ttl
2.7:  64.08% vs 35.92%,   103 ttl
2.8:  64.42% vs 35.58%,   104 ttl
2.9:  64.42% vs 35.58%,   104 ttl
3.0:  64.42% vs 35.58%,   104 ttl
3.1:  64.42% vs 35.58%,   104 ttl
3.2:  63.81% vs 36.19%,   105 ttl
3.3:  63.81% vs 36

# Test different values for sma_minutes

In [77]:
sma_minutes = [3, 5, 10, 15, 20, 30]
days_lookback = 5
threshold_start = 0.5
threshold_end = 4
threshold_step = 0.1
for sma_minute in sma_minutes:
    start = time.time()
    print(f'PROCESSING sma_minutes -> {sma_minute}')
    bars_output = compute_rvol(bars, days_lookback=days_lookback, days_lookback_buffer=days_lookback*2, rvol_sma_minutes=sma_minute)
    _df = populate_rvol(df, bars_output, rvol_days_lookback=days_lookback, rvol_sma_minutes=sma_minute)
    df_good = _df[_df['r'] > 0]
    df_bad = _df[_df['r'] < 0]

    threshold = threshold_start
    while threshold < threshold_end:
        c1 = f'rvol_{days_lookback}_SMA_{sma_minute}min'
        _df_good = df_good[df_good[c1] < threshold]
        _df_bad = df_bad[df_bad[c1] < threshold]
        print(f'{threshold:0.1f}:  {(_df_good.shape[0] / (_df_good.shape[0] + _df_bad.shape[0])):.2%} vs {(_df_bad.shape[0] / (_df_good.shape[0] + _df_bad.shape[0])):.2%},   {(_df_good.shape[0] + _df_bad.shape[0]):3} ttl')
        threshold = round(threshold + threshold_step, 2)
    
    print(f'      - {(time.time() - start):.1f} seconds elapsed')


PROCESSING sma_minutes -> 3
0.5:  66.67% vs 33.33%,     3 ttl
0.6:  60.00% vs 40.00%,    10 ttl
0.7:  66.67% vs 33.33%,    18 ttl
0.8:  68.97% vs 31.03%,    29 ttl
0.9:  69.23% vs 30.77%,    39 ttl
1.0:  69.39% vs 30.61%,    49 ttl
1.1:  66.67% vs 33.33%,    63 ttl
1.2:  66.67% vs 33.33%,    72 ttl
1.3:  67.95% vs 32.05%,    78 ttl
1.4:  67.07% vs 32.93%,    82 ttl
1.5:  67.82% vs 32.18%,    87 ttl
1.6:  68.89% vs 31.11%,    90 ttl
1.7:  68.48% vs 31.52%,    92 ttl
1.8:  67.71% vs 32.29%,    96 ttl
1.9:  67.71% vs 32.29%,    96 ttl
2.0:  68.04% vs 31.96%,    97 ttl
2.1:  68.04% vs 31.96%,    97 ttl
2.2:  68.04% vs 31.96%,    97 ttl
2.3:  66.67% vs 33.33%,    99 ttl
2.4:  66.00% vs 34.00%,   100 ttl
2.5:  65.35% vs 34.65%,   101 ttl
2.6:  64.71% vs 35.29%,   102 ttl
2.7:  64.08% vs 35.92%,   103 ttl
2.8:  64.08% vs 35.92%,   103 ttl
2.9:  64.08% vs 35.92%,   103 ttl
3.0:  64.08% vs 35.92%,   103 ttl
3.1:  64.08% vs 35.92%,   103 ttl
3.2:  64.08% vs 35.92%,   103 ttl
3.3:  63.46% vs 36.5

In [58]:
df_good['rvol_5'].mean()

1.103853911750953

In [59]:
df_bad['rvol_5'].mean()

1.5180490633584898

# ITERATE AND TRY A RANGE OF THRESHOLDS

In [36]:
df = _df
df = df[(df['parameter_buffer'] == 0.1) & (df['parameter_stop'] == 0.8) & (df['parameter_target'] == 1.1) & (df['distance'] > 0.25)]
df = df[df['direction'] == 'short']

df_good = df[df['r'] > 0]
df_bad = df[df['r'] < 0]

print(f'df_good - {(df_good.shape[0] / df.shape[0]):.2%}')
print(f' df_bad - {(df_bad.shape[0] / df.shape[0]):.2%}')
print(f'    ttl - {(df_good.shape[0] + df_bad.shape[0])}')
print()

threshold = 0.5
while threshold < 4:
    print(threshold)
    _df_good = df_good[df_good['rvol_5_SMA_15min'] < threshold]
    _df_bad = df_bad[df_bad['rvol_5_SMA_15min'] < threshold]
    print(f'  df_good - {(_df_good.shape[0] / (_df_good.shape[0] + _df_bad.shape[0])):.2%}')
    print(f'   df_bad - {(_df_bad.shape[0] / (_df_good.shape[0] + _df_bad.shape[0])):.2%}')
    print(f'      ttl - {(_df_good.shape[0] + _df_bad.shape[0])}')
    print()
    threshold = round(threshold + 0.1, 2)

# select_qty = df_good[df_good['rvol_5'] > 2].shape[0]
# print(f'df_good - {(select_qty / df_good.shape[0]):.2%}')

# select_qty = df_bad[df_bad['rvol_5'] > 2].shape[0]
# print(f'df_bad - {(select_qty / df_bad.shape[0]):.2%}')


df_good - 49.15%
 df_bad - 50.00%
    ttl - 117

0.5
  df_good - 14.29%
   df_bad - 85.71%
      ttl - 7

0.6
  df_good - 27.27%
   df_bad - 72.73%
      ttl - 11

0.7
  df_good - 47.83%
   df_bad - 52.17%
      ttl - 23

0.8
  df_good - 44.74%
   df_bad - 55.26%
      ttl - 38

0.9
  df_good - 44.00%
   df_bad - 56.00%
      ttl - 50

1.0
  df_good - 42.65%
   df_bad - 57.35%
      ttl - 68

1.1
  df_good - 42.86%
   df_bad - 57.14%
      ttl - 77

1.2
  df_good - 44.71%
   df_bad - 55.29%
      ttl - 85

1.3
  df_good - 46.81%
   df_bad - 53.19%
      ttl - 94

1.4
  df_good - 47.42%
   df_bad - 52.58%
      ttl - 97

1.5
  df_good - 47.52%
   df_bad - 52.48%
      ttl - 101

1.6
  df_good - 48.08%
   df_bad - 51.92%
      ttl - 104

1.7
  df_good - 49.06%
   df_bad - 50.94%
      ttl - 106

1.8
  df_good - 48.18%
   df_bad - 51.82%
      ttl - 110

1.9
  df_good - 48.18%
   df_bad - 51.82%
      ttl - 110

2.0
  df_good - 47.75%
   df_bad - 52.25%
      ttl - 111

2.1
  df_good - 48

In [34]:
select_qty = df_good[df_good['rvol_5'] > 1].shape[0]
print(f'df_good - {(select_qty / df_good.shape[0]):.2%}')

select_qty = df_bad[df_bad['rvol_5'] > 1].shape[0]
print(f'df_bad - {(select_qty / df_bad.shape[0]):.2%}')


df_good - 41.67%
df_bad - 45.07%


# MISC

In [ ]:
def generate_dates(start_date, end_date, start_hour=9, start_minute=30, end_hour=12, end_minute=0):
    dates_list = []
    for d in pandas.date_range(start=start_date, end=end_date):
        if d.weekday() > 4:
            continue
        start = pytz.timezone('US/Eastern').localize(d.replace(hour=start_hour, minute=start_minute)).astimezone(pytz.utc).isoformat()
        end = pytz.timezone('US/Eastern').localize(d.replace(hour=end_hour, minute=end_minute)).astimezone(pytz.utc).isoformat()
        dates_list.append((d.strftime("%Y-%m-%d"), start, end))
    return dates_list

In [ ]:
def get_quotes(ticker, start, end, name=None):
    quotes = alpaca.get_quotes([ticker], start, end).df
    quotes = quotes.loc[~quotes.index.duplicated(keep='last')]
    
    if not quotes.empty:
        quotes = quotes[quotes['ask_price'] - quotes['bid_price'] > 0.0]
    
    return quotes


In [ ]:
start_date = '2020-12-10'
end_date = '2022-12-14'

dates = generate_dates(start_date, end_date)


In [ ]:
for i, date in enumerate(dates):
    print(f'{i} / {len(dates)}')
        
    date_string = date[0]
    
    from_iso = date[1]
    to_iso = date[2]
    
    q = get_quotes('SPY', from_iso, to_iso)
    q.to_pickle(f'quotes/SPY_{date_string}.pickle')
